In [3]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import nltk
import pandas as pd
import numpy as np
import re

In [4]:
conn = sqlite3.connect("../../diplom.db")

In [7]:
kom = pd.read_sql("""SELECT ID, TRANSCRIPT FROM TRANSCRIPTS WHERE SOURCE = 4""", conn)

In [11]:
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem

pattern = re.compile("[^a-zA-Z0-9а-яА-ЯёЁ-]+")
stopwords_rus = stopwords.words('russian')
stemmer = Mystem()


def text_cleaner(text):
    clean_text = ' '.join(list(filter(None, [i.strip() for i in re.sub(pattern, ' ', text).lower().split(' ') if
                                             i not in stopwords_rus])))
    out = ' '.join([i for i in stemmer.lemmatize(clean_text) if i not in (' ', '\n')])
    return out


In [ ]:
kom["CLEAN_TEXT"] = kom.TRANSCRIPT.apply(text_cleaner)

In [ ]:
kom["DEMOCRACY_COUNT"] = kom["CLEAN_TEXT"].apply(lambda text: text.count("демократия"))

In [ ]:
kom.to_sql("KOMMERSANT", conn, index=False)